In [ ]:
%%capture

!pip install gdown gpxpy

In [ ]:
%%capture

import gdown

DRIVE_FOLDER = 'https://drive.google.com/drive/folders/10Q8FkfqAhp8ShVB_AqwDvCrNzU69NFId?usp=sharing'
gdown.download_folder(DRIVE_FOLDER, quiet=True)

In [ ]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

In [ ]:
osmtracker_df = parse_gpx('/content/polarflow-vs-osmtracker/osmtracker.gpx')
osmtracker_df.head()

In [ ]:
polarflow_df = parse_gpx('/content/polarflow-vs-osmtracker/polarflow.gpx')
polarflow_df.head()

In [ ]:
import folium

m = folium.Map(
    location=(
      osmtracker_df['lat'].mean(),
      osmtracker_df['lon'].mean(),
    ),
    zoom_start=14,
)

def add_to_map(df, color):
    coordinates = df[['lat', 'lon']].values.tolist()
    folium.PolyLine(locations=coordinates, color=color, weight=5, opacity=1).add_to(m)

add_to_map(osmtracker_df, 'black')
add_to_map(polarflow_df, 'orange')
m

In [ ]:
osmtracker_df.shape

In [ ]:
polarflow_df.shape

In [ ]:
import numpy as np

def get_time_deltas(df):
    df_sorted = df.sort_values(by='time')

    time_deltas = df_sorted['time'].diff().dropna()
    time_deltas_seconds = time_deltas.dt.total_seconds()

    p10 = np.percentile(time_deltas_seconds, 10)
    p50 = np.percentile(time_deltas_seconds, 50)
    p90 = np.percentile(time_deltas_seconds, 90)
    p99 = np.percentile(time_deltas_seconds, 99)

    print(f"P10: {p10}")
    print(f"P50: {p50}")
    print(f"P90: {p90}")
    print(f"P99: {p99}")

In [ ]:
get_time_deltas(osmtracker_df)

In [ ]:
get_time_deltas(polarflow_df)